In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

# User profile

In [ ]:
ratings_df = pd.read_csv('/data/kakao_data/ratings_two.csv')
print(len(ratings_df))

23308


In [ ]:
ratings_df[ratings_df['user_id'] =='10l9o2l']

,name,address,user_id,rating,review,cafe
540,안다르커피,창천동,10l9o2l,5.0,분위기가 공부하기에 딱 좋구 디저트도 맛있어요\n사장님도 친절하세요! ㅎㅎ,안다르커피 창천동
541,카페노티드 안국점,재동,10l9o2l,2.0,캐릭터는 귀여운데 도넛이 너무 느끼함 욱……,카페노티드 안국점 재동


In [ ]:
users = pd.read_csv('/data/kakao_data/users.csv')
users[users['user_id']=='10l9o2l']

,user_id,user_name,user_review_num,user_rating_avg
24111,10l9o2l,롱롱,23,4.0


# Content-Based Filtering (ML)

In [ ]:
public_df = pd.read_csv("/data/public_data/publicdata_cafe_lastdrop.csv")
print(len(public_df))

19797


In [ ]:
## CBF의 경우, 이미 갔던 카페들 중 카페 안다르만 제거하고 추천하였음.
def recommend_cafe(df_svd_preds, user_id, ori_ratings_df, num_recommendations=5):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_svd_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']


    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')

    # 이미 갔던 곳 제외하기 위한 코드 추가
    user_data.reset_index(inplace=True, drop=True)
    recommendations = recommendations[recommendations.index!='안다르커피 창천동']

    recommendations = pd.merge(recommendations, public_df.set_index('cafe')[['시군구명','도로명주소']], left_index=True,right_index=True,how='left')
    recommendations.drop('cafe',axis=1,inplace=True)

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [ ]:
def recommend_cafe_address(df_svd_preds, user_id, ori_ratings_df, num_recommendations=5, address='영등포구'):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_svd_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']


    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')

    # 이미 갔던 곳 제외하기 위한 코드 추가
    user_data.reset_index(inplace=True, drop=True)
    recommendations = recommendations[recommendations.index!='안다르커피 창천동']
    
    recommendations = pd.merge(recommendations, public_df.set_index('cafe')[['시군구명','도로명주소']], left_index=True,right_index=True,how='left')
    recommendations.drop('cafe',axis=1,inplace=True)

    ## address 부분
    recommendations = recommendations[recommendations['시군구명']==address]

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

## RandomForest Regressor

### ratings 2개 이상

In [ ]:
rfr_full = pd.read_csv('저장경로/CBF_total_two_RFR.csv')

In [ ]:
rfr_full = rfr_full.rename(columns = {'Unnamed: 0':'user_id'})
rfr_full.set_index('user_id',inplace=True)

In [ ]:
already_rated, predictions = recommend_cafe(rfr_full, user_id='12c6sec', ori_ratings_df=ratings_df,num_recommendations=10)

In [ ]:
already_rated

,cafe,rating
0,943킹스크로스 서교동,2.0
1,ATOZCAFE 창천동,5.0
2,가빈커피 창천동,5.0
3,닥터로빈 이대점 대현동,4.0
4,벤치커피스튜디오 대현동,5.0
5,솔트포레스트 연남동,5.0
6,스튜디오웝 대신동,5.0
7,안다르커피 창천동,5.0
8,앤티크커피 동교동,5.0
9,연남방앗간 연남동,4.0


In [ ]:
predictions

,Predictions,시군구명,도로명주소
원앤온리커피 대현동,5.00,서대문구,서울특별시 서대문구 이화여대7길 31-4
맨케이브커피 동선동2가,4.98,성북구,서울특별시 성북구 보문로34길 94
이나더베어 합정동,4.98,마포구,서울특별시 마포구 독막로2길 31
카페 도르래 삼성동,4.98,강남구,서울특별시 강남구 삼성로116길 5
이디야커피 명지대점 남가좌동,4.96,서대문구,서울특별시 서대문구 거북골로 33
핸드릭스커피로스터스 동교동,4.96,마포구,서울특별시 마포구 월드컵북로4길 20
알티프로젝트 동숭동,4.96,종로구,서울특별시 종로구 동숭3길 26-1
창신단길 창신동,4.96,종로구,서울특별시 종로구 창신길 16
데스칸사르 논현동,4.96,강남구,서울특별시 강남구 언주로113길 17
앳더블랑 흑석동,4.96,동작구,서울특별시 동작구 흑석로13가길 33


In [ ]:
already_rated, predictions = recommend_cafe_address(rfr_full, user_id='12c6sec', ori_ratings_df=ratings_df,num_recommendations=10,
                                                    address='서대문구')

In [ ]:
predictions

,Predictions,시군구명,도로명주소
원앤온리커피 대현동,5.00,서대문구,서울특별시 서대문구 이화여대7길 31-4
이디야커피 명지대점 남가좌동,4.96,서대문구,서울특별시 서대문구 거북골로 33
ATOZCAFE 창천동,4.94,서대문구,서울특별시 서대문구 연세로4길 30
벤치커피스튜디오 대현동,4.94,서대문구,서울특별시 서대문구 연세로2마길 14
헤일우드 홍제동,4.92,서대문구,서울특별시 서대문구 통일로34길 7
원오브씨 창천동,4.92,서대문구,서울특별시 서대문구 신촌로3길 5
낙랑파라 아현점 북아현동,4.92,서대문구,서울특별시 서대문구 신촌로31길 20
카페 피에노 북아현동,4.88,서대문구,서울특별시 서대문구 신촌로37길 16
가빈커피 창천동,4.88,서대문구,서울특별시 서대문구 명물길 49
롱앤쇼트 홍제동,4.88,서대문구,서울특별시 서대문구 홍제내2나길 46


## SVR

### ratings 2개 이상

In [ ]:
svr_full = pd.read_csv('저장경로/CBF_total_two_SVR.csv')

In [ ]:
svr_full.head()

,Unnamed: 0,버블베어 방화동,커피랩 서교동,커핀그루나루신천점 신천점 잠실동,이디야커피 신길역점 신길동,커피빈 코리아청담성당점 청담동,스타벅스 동숭로아트점 동숭동,가비애 서교동,훈고링고브레드 성산동,스타벅스남부터미널2점 남부터미널2점 서초동,...,이디야커피 선릉사거리점 역삼동,이디야커피 당곡사거리점 봉천동,스타벅스 삼성현대힐점 삼성동,이디야커피 문정테라타워점 문정동,스타벅스 안국점 재동,이디야커피 동자동점 동자동,커피나무 보라매역점 신길동,이디야커피 논골사거리점 금호동1가,퍼블릭맨션 목동,이디야커피 굽은다리역점 천호동
0,100ck0j,2.366667,2.366667,2.366667,2.366667,2.366667,2.366691,2.366667,2.366667,2.367095,...,2.366667,2.366667,2.544408,2.366667,2.366667,2.366667,2.366667,2.366667,2.366667,2.366667
1,100i22n,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550000,3.550002,...,3.550000,3.550000,3.550003,3.550000,3.550026,3.550000,3.550000,3.550000,3.550000,3.550000
2,100ooji,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
3,100q04c,3.590065,3.590065,3.590065,3.590065,3.590065,3.590092,3.590065,3.590065,3.592980,...,3.590065,3.590065,3.590066,3.590065,3.596362,3.590065,3.590065,3.590065,3.590065,3.590065
4,100rluq,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.999985,...,3.000000,3.000000,2.999999,3.000000,2.999971,3.000000,3.000000,3.000000,3.000000,3.000000


In [ ]:
svr_full = svr_full.rename(columns = {'Unnamed: 0':'user_id'})
svr_full.set_index('user_id',inplace=True)

In [ ]:
already_rated, predictions = recommend_cafe(svr_full, user_id='12c6sec', ori_ratings_df=ratings_df,num_recommendations=10)

In [ ]:
predictions

,Predictions,시군구명,도로명주소
앤티크커피 동교동,4.900342,마포구,서울특별시 마포구 연희로 25-1
스튜디오웝 대신동,4.900329,서대문구,서울특별시 서대문구 성산로 539
웨스트커프하우스 논현동,4.900329,강남구,서울특별시 강남구 도산대로54길 10
벤치커피스튜디오 대현동,4.899942,서대문구,서울특별시 서대문구 연세로2마길 14
솔트포레스트 연남동,4.899942,마포구,서울특별시 마포구 동교로46길 18-3
ATOZCAFE 창천동,4.899942,서대문구,서울특별시 서대문구 연세로4길 30
원앤온리커피 대현동,4.899761,서대문구,서울특별시 서대문구 이화여대7길 31-4
저스트단비 서교동,4.899761,마포구,서울특별시 마포구 잔다리로6길 30
가빈커피 창천동,4.899753,서대문구,서울특별시 서대문구 명물길 49
카페수달 안국동,4.524353,종로구,서울특별시 종로구 율곡로 49-14


In [ ]:
already_rated, predictions = recommend_cafe_address(svr_full, user_id='12c6sec', ori_ratings_df=ratings_df,num_recommendations=10,
                                                    address='서대문구')

In [ ]:
predictions

,Predictions,시군구명,도로명주소
스튜디오웝 대신동,4.900329,서대문구,서울특별시 서대문구 성산로 539
벤치커피스튜디오 대현동,4.899942,서대문구,서울특별시 서대문구 연세로2마길 14
ATOZCAFE 창천동,4.899942,서대문구,서울특별시 서대문구 연세로4길 30
원앤온리커피 대현동,4.899761,서대문구,서울특별시 서대문구 이화여대7길 31-4
가빈커피 창천동,4.899753,서대문구,서울특별시 서대문구 명물길 49
스타벅스 가재울뉴타운점 남가좌동,4.433870,서대문구,서울특별시 서대문구 가재울미래로 2
스타벅스 신촌아트레온점 창천동,4.433538,서대문구,서울특별시 서대문구 신촌로 129
투썸플레이스충정로역점 충정로역점 충정로3가,4.433537,서대문구,서울특별시 서대문구 서소문로 27
스타벅스신촌명물거리점 신촌명물거리점 창천동,4.433535,서대문구,서울특별시 서대문구 명물길 24
마실ing 홍은동,4.433535,서대문구,서울특별시 서대문구 모래내로 357
